# PennyLane: Quantum Data embedding Methods for Quantum Machine Learning

Link Website: https://medium.datadriveninvestor.com/all-about-data-encoding-for-quantum-machine-learning-2a7344b1dfef
- https://arxiv.org/pdf/2101.11020.pdf (Paper by: Maria Schuld)


![](https://miro.medium.com/max/549/1*dqxQbP8iJ8g6ZMaEBcELlg.png)


## Preprocessing and state preparation:

![](https://miro.medium.com/max/700/1*VFsEP9cVIkIJS_vo6ufujQ.png)


In [71]:
from pennylane import numpy as np

# helper functions
# Function to print the output
def printTheArray(arr, n):
 
    for i in range(0, n):
        print(arr[i], end = " ")
     
    print()
 
# Function to generate all binary strings
def generateAllBinaryStrings(n, arr, i):
 
    if i == n:
        printTheArray(arr, n)
        return
     
    # First assign "0" at ith position
    # and try for all other permutations
    # for remaining positions
    arr[i] = 0
    generateAllBinaryStrings(n, arr, i + 1)
 
    # And then assign "1" at ith position
    # and try for all other permutations
    # for remaining positions
    arr[i] = 1
    generateAllBinaryStrings(n, arr, i + 1)

## Basis encoding

In [78]:
# Basis encoding

x_1 = [5 ,6]
x_2 = [4 ,1]

# dec, binary, qstate
print('x_1 =>\n decimal: [{}, {}]'.format(x_1[0],x_1[1])
      , '\n binary: [{:03b}, {:03b}]'.format(x_1[0],x_1[1])
      , '\n basis encoded quantum state: |{:03b}{:03b}>'.format(x_1[0],x_1[1])
      , '\n' )

print('x_2 =>\n decimal: [{}, {}]'.format(x_2[0],x_2[1])
      , '\n binary: [{:03b}, {:03b}]'.format(x_2[0],x_2[1])
      , '\n basis encoded quantum state: |{:03b}{:03b}>'.format(x_2[0],x_2[1]) )

x_1 =>
 decimal: [5, 6] 
 binary: [101, 110] 
 basis encoded quantum state: |101110> 

x_2 =>
 decimal: [4, 1] 
 binary: [100, 001] 
 basis encoded quantum state: |100001>


In [73]:
# pennylane code
import pennylane as qml
from pennylane import numpy as np

# import the template
from pennylane.templates.embeddings import BasisEmbedding

# quantum device where you want to run and how many Qubits
dev = qml.device('default.qubit', wires=6)

@qml.qnode(dev)
def circuit(data):
    for i in range(6):
        qml.Hadamard(i)
    for i in range(len(data)):
        BasisEmbedding(features=data[i], wires=range(6),do_queue=True)
    return  qml.state()

data=[[1,0,1,1,1,0],
      [1,0,0,0,0,1]]

circuit(data)

print(circuit.draw(show_all_wires=True))

# Watch out when intepreting the drawing
# => The last X-gate on qubit #5 is should be on the right hand side (3rd gate position)
# pennylane drawings don't keep paddings


 0: ──H──X──X──╭┤ State 
 1: ──H────────├┤ State 
 2: ──H──X─────├┤ State 
 3: ──H──X─────├┤ State 
 4: ──H──X─────├┤ State 
 5: ──H──X─────╰┤ State 



## Amplitude encoding

![](https://miro.medium.com/max/700/1*GL5v5y2uokrTVo3_Pkr3bw.png)

In [79]:
# Amplitude encoding

data = np.array([6,-12.5,11.15,7])

# normalization
normalization = np.linalg.norm(data)
X_norm = (data.T / normalization).T

print( 'Amplitude encoding: {}/{norm:.2f}|00> -  {}/{norm:.2f}|01> + {}/{norm:.2f}|10> + {}/{norm:.2f}|11> '.format(data[0], data[1], data[2], data[3], norm=normalization) 
       , "\nAmplitudes: ", X_norm)

Amplitude encoding: 6.0/19.12|00> -  -12.5/19.12|01> + 11.15/19.12|10> + 7.0/19.12|11>  
Amplitudes:  [ 0.31380835 -0.65376739  0.58316051  0.36610974]


In [75]:
# pennylane code
# import the template
from pennylane.templates.embeddings import AmplitudeEmbedding

# quantum device where you want to run and how many Qubits

dev = qml.device('default.qubit', wires=2)
@qml.qnode(dev)
def circuit(data, probs=False):
    AmplitudeEmbedding(features=data, wires=range(2),normalize=True)
    if probs:
      return  qml.probs(wires=range(2))
    return qml.state()

data = [6, -12.5, 11.15, 7]
circuit(data)

print(circuit.draw(show_all_wires=True))
print("probs:")
print(circuit(data, True))

# Print all binary strings
qubits=2
print("\nBinary permutations for {}:".format(qubits))
generateAllBinaryStrings(qubits, [None] * qubits, 0)

 0: ──╭QubitStateVector(M0)──╭┤ State 
 1: ──╰QubitStateVector(M0)──╰┤ State 
M0 =
[ 0.31380835+0.j -0.65376739+0.j  0.58316051+0.j  0.36610974+0.j]

probs:
[0.09847568 0.4274118  0.34007618 0.13403634]

Binary permutations for 2:
0 0 
0 1 
1 0 
1 1 


## Angle encoding

![](https://miro.medium.com/max/700/1*zL56tgxQZNlT-30jcxYlug.png)

In [76]:
# Angle encoding

data = np.array([[6,-12.5,11.15,7],[8,9.5,-11,-5],[5,0.5,8,-7]])

# x/2
X_half = (data.T / 2).T

print(  "\nValues:\n", X_half)


Values:
 [[ 3.    -6.25   5.575  3.5  ]
 [ 4.     4.75  -5.5   -2.5  ]
 [ 2.5    0.25   4.    -3.5  ]]


In [77]:
# pennylane code 

# import the template
from pennylane.templates.embeddings import AngleEmbedding
dev = qml.device('default.qubit', wires=4)
@qml.qnode(dev)
def circuit(data, probs=False):
    for i in range(4):
        qml.Hadamard(i)
    for i in range(len(data)):
        AngleEmbedding(features=data[i], wires=range(4),rotation='Y')
    if probs:
      return  qml.probs(wires=range(4))
    return  qml.state()
    

data = np.array([[6,-12.5,11.15,7],[8,9.5,-11,-5],[5,0.5,8,-7]])

print(circuit(data))
print()
print(circuit.draw(show_all_wires=True))
print("probs:")
print(circuit(data, True))

# Print all binary strings
qubits=4
print("\nBinary permutations for {}:".format(qubits))
generateAllBinaryStrings(qubits, [None] * qubits, 0)

[ 0.01231807+0.j  0.08506659+0.j -0.08261645+0.j -0.57053559+0.j
 -0.00617374+0.j -0.04263481+0.j  0.04140682+0.j  0.28594865+0.j
  0.01432609+0.j  0.09893363+0.j -0.09608408+0.j -0.66354082+0.j
 -0.00718014+0.j -0.04958488+0.j  0.0481567 +0.j  0.33256225+0.j]

 0: ──H──RY(6)──────RY(8)────RY(5)────╭┤ State 
 1: ──H──RY(-12.5)──RY(9.5)──RY(0.5)──├┤ State 
 2: ──H──RY(11.2)───RY(-11)──RY(8)────├┤ State 
 3: ──H──RY(7)──────RY(-5)───RY(-7)───╰┤ State 

probs:
[1.51734964e-04 7.23632517e-03 6.82547810e-03 3.25510862e-01
 3.81150307e-05 1.81772710e-03 1.71452446e-03 8.17666287e-02
 2.05236913e-04 9.78786299e-03 9.23215070e-03 4.40286422e-01
 5.15544410e-05 2.45866007e-03 2.31906805e-03 1.10597650e-01]

Binary permutations for 4:
0 0 0 0 
0 0 0 1 
0 0 1 0 
0 0 1 1 
0 1 0 0 
0 1 0 1 
0 1 1 0 
0 1 1 1 
1 0 0 0 
1 0 0 1 
1 0 1 0 
1 0 1 1 
1 1 0 0 
1 1 0 1 
1 1 1 0 
1 1 1 1 
